# OpenAI SwarmでWeb検索、YouTube検索のAIエージェントを作成し、実行する

In [1]:
# ライブラリをインポート
import os
from getpass import getpass
from pprint import pprint
from swarm import Swarm, Agent
from openai import OpenAI
from duckduckgo_search import DDGS
import pandas as pd
from googleapiclient.discovery import build

In [ ]:
# OpenAIのAPIキーを入力
openai_api_key = getpass(prompt="Please enter OpenAI API key:")

In [ ]:
# YouTube Data APIのAPIキーを入力
youtube_data_api_key = getpass(prompt="Please enter YouTube Data API key:")

In [4]:
# OpenAIのAPIキーを環境変数を設定
os.environ["OPENAI_API_KEY"] = openai_api_key

In [5]:
# Swarmクライアントを初期化
client = Swarm()

In [6]:
# Web検索を行う関数
def search_web(keyword: str) -> pd.DataFrame:
    try:
        with DDGS() as ddgs:
            results = list(ddgs.text(
                keywords=keyword, 
                region="jp-jp", 
                safesearch="off", 
                timelimit=None, 
                max_results=10
            ))
        
        # データをデータフレームに変換
        data = [
            {
                "タイトル": result.get("title", ""),
                "URL": result.get("href", ""),
                "本文": result.get("body", "")
            } 
            for result in results
        ]
        df = pd.DataFrame(data)
        df.index = df.index + 1
        
        return df
    
    except Exception as e:
        print(f"Error: {e}")
        return pd.DataFrame()  # エラー時には空のデータフレームを返す

In [ ]:
# Web検索を行う
# search_web("宮崎グルメ")

In [7]:
# YouTube動画を検索する関数
def search_youtube_videos(query: str) -> pd.DataFrame:
    try:
        youtube = build("youtube", "v3", developerKey=youtube_data_api_key)

        # 動画検索
        request = youtube.search().list(
            part="snippet",
            q=query,
            type="video",
            maxResults=10
        )
        response = request.execute()

        # 結果をデータフレームに変換
        data = []
        for item in response.get("items", []):
            video_id = item["id"]["videoId"]
            title = item["snippet"]["title"]
            description = item["snippet"]["description"]
            channel_title = item["snippet"]["channelTitle"]
            publish_time = item["snippet"]["publishedAt"]

            data.append({
                "タイトル": title,
                "説明": description,
                "チャンネル名": channel_title,
                "公開日時": publish_time,
                "URL": f"https://www.youtube.com/watch?v={video_id}"
            })

        df = pd.DataFrame(data)
        df.index = df.index + 1

        return df

    except Exception as e:
        print(f"Error: {e}")
        return pd.DataFrame()  # エラー時は空のデータフレームを返す

In [ ]:
# YouTube動画を検索
# search_youtube_videos("宮崎グルメ")

In [ ]:
# トリアージエージェントへのインストラクション
triage_agent_instructions = """
あなたはユーザーの要求を適切に判定できるトリアージの専門家です。ユーザーの要求に応じて、適切なエージェントに転送してください。
YouTubeの検索が必要な場合は、youtube_search_agentに会話を転送してください。
Web検索が必要な場合は、web_search_agentに会話を転送してください。
転送が不要で、直接質問に回答できる場合は回答してください。
"""

In [9]:
# トリアージエージェント
triage_agent = Agent(
    name="トリアージエージェント",
    model="gpt-4o-mini",
    instructions=triage_agent_instructions
)

In [10]:
# Web検索エージェントへのインストラクション
web_search_agent_instructions = """
あなたはWeb検索の専門家です。ユーザーからキーワードでWeb検索を要求されたら、その結果をデータフレームで提供してください。検索については、search_webの関数を使用してください。
Web検索が不要な場合は、triage_agentに会話を転送してください。
YouTube検索が必要な場合は、youtube_search_agentに会話を転送してください。
"""

In [11]:
# Web検索エージェント
web_search_agent = Agent(
    name="Web検索エージェント",
    model="gpt-4o-mini",
    instructions=web_search_agent_instructions,
    functions=[search_web]
)

In [12]:
# YouTube検索エージェントへのインストラクション
youtube_search_agent_instructions = """
あなたはYouTube検索の専門家です。ユーザーからキーワードでYouTube検索を要求されたら、その結果をデータフレームで提供してください。検索については、search_youtube_videosの関数を使用してください。
YouTube検索が不要な場合は、triage_agentに会話を転送してください。
Web検索が必要な場合は、web_search_agentに会話を転送してください。
"""

In [13]:
# YouTube検索エージェント
youtube_search_agent = Agent(
    name="YouTube検索エージェント",
    model="gpt-4o-mini",
    instructions=youtube_search_agent_instructions,
    functions=[search_youtube_videos]
)

In [ ]:
# Web検索エージェントに転送する関数
def transfer_to_web_search_agent():
    print("Web検索エージェントに転送します")
    return web_search_agent

# YouTube検索エージェントに転送する関数
def transfer_to_youtube_search_agent():
    print("YouTube検索エージェントに転送します")
    return youtube_search_agent

# トリアージエージェントに転送に転送する関数
def transfer_back_to_triage_agent():
    print("トリアージエージェントに転送します")
    return triage_agent

# triage_agentにWeb検索エージェント、YouTube検索エージェントに転送する関数を指定
triage_agent.functions = [transfer_to_web_search_agent, transfer_to_youtube_search_agent]

# web_search_agentにトリアージエージェントに転送に転送する関数を追加
web_search_agent.functions.append(transfer_back_to_triage_agent)

# web_search_agentにYouTube検索エージェントに転送に転送する関数を追加
web_search_agent.functions.append(transfer_to_youtube_search_agent)

# youtube_search_agentにトリアージエージェントに転送に転送する関数を追加
youtube_search_agent.functions.append(transfer_back_to_triage_agent)

# youtube_search_agentにWeb検索エージェントに転送に転送する関数を追加
youtube_search_agent.functions.append(transfer_to_web_search_agent)

In [15]:
# エージェントを呼び出して、実行 (Web検索)
response_web_search = client.run(
    agent=triage_agent,
    messages=[{"role": "user", "content": "宮崎グルメというキーワードでWeb検索をしてください。"}]
)

# 結果を確認
print(response_web_search.messages[-1]["content"])

Web検索エージェントに転送します
以下は「宮崎グルメ」に関するWeb検索結果です。

| タイトル                                                                                       | URL                                                | 本文                                                                                                     |
|-----------------------------------------------------------------------------------------------|---------------------------------------------------|----------------------------------------------------------------------------------------------------------|
| 宮崎代表＆発祥グルメ!宮崎市に訪れたらコレ絶対食べてみて! | [リンク](https://www.miyazaki-city.tourism.or.jp/feature/) | 宮崎県の豊富な食材と伝統のグルメを紹介する特集ページです。地鶏の炭火焼きや宮崎牛、チキン南蛮...                  |
| 宮崎旅行で食べたい!名物グルメが食べられるお店18選                                           | [リンク](https://s.tabelog.com/smartphone/matome/15134/) | 宮崎県は南九州に位置し、年間を通して暖かく晴れた日が多い地域。降水量も多いため、植物もよく育...                |
| 宮崎の美味しいご当地グルメ10選。絶対食べたい名物グルメはコレ!おすすめ店も紹介                         | [リンク](https://www.ja

In [16]:
# エージェントを呼び出して、実行 (Web検索)
response_youtube_search = client.run(
    agent=triage_agent,
    messages=[{"role": "user", "content": "宮崎グルメというキーワードでYouTube検索をしてください。"}]
)

# 結果を確認
print(response_youtube_search.messages[-1]["content"])

YouTube検索エージェントに転送します
以下は「宮崎グルメ」というキーワードでのYouTube検索結果です。

| タイトル                                                                 | 説明                                                                                                       | チャンネル名                             | 公開日時               | URL                                               |
|--------------------------------------------------------------------------|------------------------------------------------------------------------------------------------------------|-----------------------------------------|------------------------|---------------------------------------------------|
| 【宮崎】宮崎出身のJAL社員がおすすめする絶品グルメ５選 #shorts            | 宮崎出身のADすけさんがイチオシの宮崎グルメ5選を紹介します！ 1軒目は[おぐら]。 宮崎と...               | JAL ふるさとプロジェクト                | 2023-09-28T11:00:08Z | [動画を見る](https://www.youtube.com/watch?v=Ar2kTScZ_bA) |
| 【宮崎グルメ】行列のできる人気店があると聞いて調査しに行ってみたら ...😳🔍 |                                                                            

In [17]:
# エージェントを呼び出して、実行 (通常の質問)
response_triage_agent = client.run(
    agent=triage_agent,
    messages=[{"role": "user", "content": "宮崎グルメでおすすめを教えてください。"}]
)

# 結果を確認
print(response_triage_agent.messages[-1]["content"])

宮崎グルメのおすすめとして、以下のものがあります。

1. **チキン南蛮** - 甘酸っぱいタルタルソースが特徴の揚げ鶏料理です。
2. **宮崎牛** - 高品質の和牛で、柔らかくて風味豊かです。ステーキや焼肉で楽しめます。
3. **日向夏** - 見た目が愛らしい果物で、甘くて酸っぱい味わいが特徴です。
4. **冷や汁** - 夏にぴったりの冷たい味噌汁で、魚や野菜を使ったさっぱりとした料理です。
5. **マンゴー** - 特に「宮崎マンゴー」は高評価で、濃厚な甘さがあります。

これらのグルメは宮崎を訪れた際にはぜひお試しください。
